In [ ]:
# default_exp ml.data

# ML data

> ML data handling functions.

TODO: create some test/data/capture_session ... so we can test end-to-end quickly

In [1]:
#export
from expoco.core import *
from pathlib import Path
import numpy as np
import pandas as pd
import json

In [2]:
#export
def viseme_dataset_from_capture_sessions(input_path='data/capture_session', glob_pattern='viseme*'):
    "Create a viseme dataset from capture session data"
    input_path, dataset_id, data = Path(input_path), now(), []
    output_path = input_path.parent/f'viseme_dataset_{dataset_id}'
    output_path.mkdir()
    metadata = dict(input_path=path_to_str(input_path), output_path=path_to_str(output_path), 
                    glob_pattern=glob_pattern, session_metadata=[], start_date=now())
    for session_path in sorted(input_path.glob(glob_pattern)):
        with open(session_path/'metadata.json') as f:
            session_metadata = json.load(f)
        column_names, expression_id = session_metadata['column_names'], session_metadata['expression_id']
        metadata['column_names'] = column_names # TODO: check that columns names match in all sessions
        del session_metadata['column_names']
        metadata['session_metadata'].append(session_metadata)
        _data = np.load(session_path/'data.npy')
        _expression_id_data = np.array([expression_id]*_data.shape[0], dtype=float)[..., None]
        _data = np.concatenate([_data, _expression_id_data], axis=1)
        data.append(_data)
    metadata['column_names'] += ['expression_id']
    with open(output_path/'metadata.json', 'w') as f: json.dump(metadata, f, indent=2)
    np.save(output_path/'data.npy', np.concatenate(data), allow_pickle=False)
    return output_path

In [3]:
# TODO: add progress bar for long running ops

In [4]:
#do_not_test
output_path = viseme_dataset_from_capture_sessions()
print('output_path', output_path)
data = np.load(output_path/'data.npy')
assert data.shape == (5000, 1405)
print('data.shape', data.shape, 'data.dtype', data.dtype)
assert data.dtype == np.dtype('float64')
with open(output_path/'metadata.json') as f:
    metadata = json.load(f)
df = pd.DataFrame(data, columns=metadata['column_names'])
df

output_path data\viseme_dataset_20211115_182351
data.shape (5000, 1405) data.dtype float64


,0x,0y,0z,1x,1y,1z,2x,2y,2z,3x,...,465x,465y,465z,466x,466y,466z,467x,467y,467z,expression_id
0,0.434740,0.629507,-0.033398,0.431403,0.566126,-0.061613,0.433960,0.585030,-0.032645,0.423069,...,0.455613,0.472091,-0.008333,0.521965,0.456807,0.008646,0.527983,0.450824,0.008500,0.0
1,0.436338,0.630942,-0.033350,0.432327,0.567964,-0.061224,0.435089,0.586540,-0.032498,0.423783,...,0.456068,0.472272,-0.007913,0.521954,0.456626,0.009275,0.527941,0.450617,0.009122,0.0
2,0.436827,0.630350,-0.034030,0.433605,0.567720,-0.061312,0.435912,0.586698,-0.032875,0.424953,...,0.457393,0.472983,-0.007093,0.523543,0.458721,0.010801,0.529732,0.452030,0.010808,0.0
3,0.439145,0.630683,-0.033820,0.436687,0.567998,-0.062186,0.438543,0.586407,-0.033073,0.427515,...,0.459443,0.472929,-0.007711,0.525564,0.457109,0.011108,0.531528,0.450968,0.011064,0.0
4,0.442390,0.629300,-0.034553,0.439859,0.567718,-0.061385,0.441522,0.586349,-0.033184,0.430574,...,0.461454,0.473333,-0.005924,0.526655,0.458836,0.013589,0.532609,0.452379,0.013715,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.416808,0.665665,-0.028273,0.410276,0.609517,-0.060504,0.414871,0.624568,-0.030215,0.401047,...,0.435961,0.504382,-0.014817,0.505507,0.482324,-0.003447,0.512044,0.474292,-0.004099,4.0
4996,0.415465,0.665505,-0.029494,0.408561,0.607583,-0.061708,0.413265,0.623417,-0.031167,0.398954,...,0.433737,0.503490,-0.014226,0.503102,0.481517,-0.002228,0.509530,0.474045,-0.002895,4.0
4997,0.412350,0.666262,-0.029774,0.404900,0.606805,-0.061258,0.410185,0.623262,-0.031080,0.396081,...,0.431835,0.504142,-0.014153,0.501963,0.483563,-0.003259,0.508464,0.475866,-0.003947,4.0
4998,0.411236,0.667711,-0.029749,0.403129,0.607193,-0.061152,0.408797,0.623940,-0.031117,0.394450,...,0.430213,0.503776,-0.014026,0.500316,0.482054,-0.003901,0.506845,0.474284,-0.004641,4.0


In [5]:
#export
def get_image_path(dataset_path, row_idx):
    dataset_path = Path(dataset_path)
    if dataset_path.name.startswith('processed'):
        # TODO: read processed metadata and find dataset path - don't just assume its the parent
        dataset_path = dataset_path.parent
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    idx = row_idx
    for session_metadata in dataset_metadata['session_metadata']:
        if idx < session_metadata['count']:
            return Path(session_metadata['path'])/f'{idx+1}.jpeg'
        idx -= session_metadata['count']

In [9]:
test_viseme_dataset = output_path # 'data/viseme_dataset_20211115_144008'
test_viseme_dataset_row_count = 5000

In [6]:
get_image_path(output_path, 400)

WindowsPath('data/capture_session/viseme_20211115_142559_2/1.jpeg')

In [ ]:
# #export
# def to_dataset_id(path):
#     path = Path(path)
#     return '_'.join(path.name.split('_')[-2:])
# TODO: remove if not used

In [7]:
#export
def read_viseme_dataset(dataset_path, to_numpy=True):
    "Return metadata and data (as a numpy array by default, pandas dataframe if `to_numpy=False`)"
    dataset_path = Path(dataset_path)
    data = np.load(dataset_path/'data.npy')
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    if to_numpy: 
        return dataset_metadata, data
    return dataset_metadata, pd.DataFrame(data, columns=dataset_metadata['column_names'])

In [8]:
_dataset_metadata, _data = read_viseme_dataset(output_path, False)
_data

,0x,0y,0z,1x,1y,1z,2x,2y,2z,3x,...,465x,465y,465z,466x,466y,466z,467x,467y,467z,expression_id
0,0.434740,0.629507,-0.033398,0.431403,0.566126,-0.061613,0.433960,0.585030,-0.032645,0.423069,...,0.455613,0.472091,-0.008333,0.521965,0.456807,0.008646,0.527983,0.450824,0.008500,0.0
1,0.436338,0.630942,-0.033350,0.432327,0.567964,-0.061224,0.435089,0.586540,-0.032498,0.423783,...,0.456068,0.472272,-0.007913,0.521954,0.456626,0.009275,0.527941,0.450617,0.009122,0.0
2,0.436827,0.630350,-0.034030,0.433605,0.567720,-0.061312,0.435912,0.586698,-0.032875,0.424953,...,0.457393,0.472983,-0.007093,0.523543,0.458721,0.010801,0.529732,0.452030,0.010808,0.0
3,0.439145,0.630683,-0.033820,0.436687,0.567998,-0.062186,0.438543,0.586407,-0.033073,0.427515,...,0.459443,0.472929,-0.007711,0.525564,0.457109,0.011108,0.531528,0.450968,0.011064,0.0
4,0.442390,0.629300,-0.034553,0.439859,0.567718,-0.061385,0.441522,0.586349,-0.033184,0.430574,...,0.461454,0.473333,-0.005924,0.526655,0.458836,0.013589,0.532609,0.452379,0.013715,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.416808,0.665665,-0.028273,0.410276,0.609517,-0.060504,0.414871,0.624568,-0.030215,0.401047,...,0.435961,0.504382,-0.014817,0.505507,0.482324,-0.003447,0.512044,0.474292,-0.004099,4.0
4996,0.415465,0.665505,-0.029494,0.408561,0.607583,-0.061708,0.413265,0.623417,-0.031167,0.398954,...,0.433737,0.503490,-0.014226,0.503102,0.481517,-0.002228,0.509530,0.474045,-0.002895,4.0
4997,0.412350,0.666262,-0.029774,0.404900,0.606805,-0.061258,0.410185,0.623262,-0.031080,0.396081,...,0.431835,0.504142,-0.014153,0.501963,0.483563,-0.003259,0.508464,0.475866,-0.003947,4.0
4998,0.411236,0.667711,-0.029749,0.403129,0.607193,-0.061152,0.408797,0.623940,-0.031117,0.394450,...,0.430213,0.503776,-0.014026,0.500316,0.482054,-0.003901,0.506845,0.474284,-0.004641,4.0


In [11]:
#export
def select_viseme_dataset_columns(dataset_metadata, data, column_names):
    return data[..., [c in column_names for c in dataset_metadata['column_names']]]

In [12]:
# TODO: fix tests
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
_cont_names = landmark_ids_to_col_names(FaceLandmarks.pointer + FaceLandmarks.mouth, [FaceLandmarks.tip_of_nose])
assert len(_cont_names) == 212
_data = select_viseme_dataset_columns(_dataset_metadata, _data, _cont_names + ['expression_id'])
assert (test_viseme_dataset_row_count, 213) == _data.shape

In [13]:
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
_data = select_viseme_dataset_columns(_dataset_metadata, _data, _cont_names[:2] + ['expression_id'])
_df = pd.DataFrame(_data, columns=_cont_names[:2] + ['expression_id'])
_df

,0x,0y,expression_id
0,0.434740,0.629507,0.0
1,0.436338,0.630942,0.0
2,0.436827,0.630350,0.0
3,0.439145,0.630683,0.0
4,0.442390,0.629300,0.0
...,...,...,...
4995,0.416808,0.665665,4.0
4996,0.415465,0.665505,4.0
4997,0.412350,0.666262,4.0
4998,0.411236,0.667711,4.0


In [14]:
#export
def make_landmarks_relative(data, to_landmarks, column_count=None):
    "Make all landmarks in `data` relative `to_landmarks`"
    coord_count = to_landmarks.shape[-1]
    assert coord_count == 2, f'to_landmarks must have 2 "coord_count". to_arr.shape={coord_count}'
    for i in range(coord_count):
        data[:, i:column_count:coord_count] -= to_landmarks[..., i][..., None]
    return data

Columns in `arr` must be ordered as pairs of landmark IDs. e.g. `'5x', '5y', '2x', '2y', '218x', '218y' ...`

- `arr` 2d np array (rows, columns)
- `to_arr` 2d np array (rows, exactly 2 columns; `x` then `y`)
    - `to_arr.shape[0]` must be the same as `arr.shape[0]` - i.e. they must have the same number of rows
    
TODO: we could allow `to_arr.shape[1]` to be 1, 2 or 3 but ... leave it as is for now - might be hard to debug data issues if we accidentaly pass 1 column when we should be passing 2.

In [15]:
_arr = np.array([[.1,.2,.3,.4,0],
                [.2,.2,.3,.4,1],
                [.2,.6,.3,.6,2]])
_to_arr = np.array([[.0,.0],
                   [.2,.4],
                   [.8,.9]])
make_landmarks_relative(_arr, _to_arr, _arr.shape[-1]-1)

array([[ 0.1,  0.2,  0.3,  0.4,  0. ],
       [ 0. , -0.2,  0.1,  0. ,  1. ],
       [-0.6, -0.3, -0.5, -0.3,  2. ]])

In [16]:
_arr = np.array([[.1,.2,.3,.4,0]])
_to_arr = np.array([[-.3,.8]])
make_landmarks_relative(_arr, _to_arr)

array([[ 0.4, -0.6,  0.6, -0.4,  0.3]])

In [17]:
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
_cont_names = landmark_ids_to_col_names(FaceLandmarks.pointer + FaceLandmarks.mouth, [FaceLandmarks.tip_of_nose])
_cont_data = select_viseme_dataset_columns(_dataset_metadata, _data, _cont_names + ['expression_id'])
_tip_of_nose_columns = landmark_ids_to_col_names([FaceLandmarks.tip_of_nose])
_tip_of_nose_data = select_viseme_dataset_columns(_dataset_metadata, _data, _tip_of_nose_columns)
_relative_data = make_landmarks_relative(_cont_data, _tip_of_nose_data, len(_cont_names))
assert (5000, 213) == _relative_data.shape
assert 0.0 == _relative_data[0][212] # check that targets have not been changed
assert 4.0 == _relative_data[4999][212] # get_image_path('20211113_114625', 'data', 1199)

In [18]:
pd.DataFrame(_relative_data, columns=_cont_names + ['expression_id'])

,0x,0y,2x,2y,5x,5y,11x,11y,12x,12y,...,409y,410x,410y,415x,415y,424x,424y,438x,438y,expression_id
0,0.003337,0.063381,0.002558,0.018904,0.000187,-0.041557,0.003528,0.072341,0.004043,0.078868,...,0.077971,0.060487,0.065190,0.039924,0.079545,0.058419,0.120741,0.027577,-0.004755,0.0
1,0.004011,0.062978,0.002762,0.018576,0.000001,-0.041924,0.004298,0.071923,0.004882,0.078523,...,0.078093,0.060569,0.064813,0.039923,0.079635,0.059342,0.120244,0.027572,-0.005052,0.0
2,0.003222,0.062630,0.002307,0.018978,0.000127,-0.042014,0.003418,0.071547,0.003867,0.078339,...,0.078261,0.059421,0.065269,0.038434,0.079994,0.057379,0.121378,0.027738,-0.004894,0.0
3,0.002458,0.062685,0.001856,0.018409,-0.000053,-0.041511,0.002591,0.071596,0.002963,0.078221,...,0.077568,0.058376,0.064308,0.037304,0.079211,0.056588,0.119681,0.027632,-0.005357,0.0
4,0.002531,0.061582,0.001664,0.018631,-0.000171,-0.041737,0.002677,0.070377,0.003028,0.077038,...,0.076824,0.057638,0.064002,0.037046,0.078517,0.055188,0.119443,0.026907,-0.004874,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.006532,0.056148,0.004595,0.015051,-0.000884,-0.042714,0.007151,0.064760,0.008168,0.071033,...,0.066363,0.066939,0.053113,0.046405,0.068419,0.066696,0.106611,0.029237,-0.007801,4.0
4996,0.006904,0.057922,0.004704,0.015834,-0.001085,-0.042871,0.007562,0.066807,0.008615,0.073324,...,0.068130,0.068074,0.054630,0.047685,0.070336,0.068065,0.110056,0.029496,-0.007753,4.0
4997,0.007450,0.059458,0.005284,0.016457,-0.000782,-0.042717,0.008120,0.068393,0.009245,0.074923,...,0.070277,0.068958,0.057018,0.048607,0.072339,0.069140,0.112344,0.029657,-0.007075,4.0
4998,0.008107,0.060518,0.005668,0.016747,-0.000820,-0.042836,0.008848,0.069468,0.010046,0.075848,...,0.070211,0.070301,0.056827,0.050314,0.072348,0.070386,0.111636,0.029608,-0.007241,4.0


In [19]:
#export
def change_target(data, from_id, to_id):
    "Change values in the last column of `data` from one ID to another"
    data[data[..., -1] == from_id] = to_id
    return data

Note: we assume that target (probably `expression_id`) is the last column in `data`

In [23]:
# TODO: fix tests
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
_unique = np.unique(_data[..., -1], return_counts=True)
assert [0., 1., 2., 3., 4.] == _unique[0].tolist()
assert [test_viseme_dataset_row_count/5]*5 == _unique[1].tolist()
change_target(_data, 4, 0)
_unique = np.unique(_data[..., -1], return_counts=True)
assert [0., 1., 2., 3.] == _unique[0].tolist()
assert [test_viseme_dataset_row_count/5*2] + [test_viseme_dataset_row_count/5]*3 == _unique[1].tolist()

In [24]:
#export
def calculate_stats(data, column_count=None):
    "Return the mean and standard deviation over columns of `data`"
    if column_count is None:
        column_count = data.shape[-1]
    _d = data[..., :column_count]
    return dict(mean=_d.mean(axis=0), std=_d.std(axis=0))

In [25]:
# TODO: fix tests
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
_data = select_viseme_dataset_columns(_dataset_metadata, _data, _cont_names[:2] + ['expression_id'])
_stats = calculate_stats(_data)
assert (3,) == _stats['mean'].shape
assert (3,) == _stats['std'].shape
_stats = calculate_stats(_data, 2)
assert (2,) == _stats['mean'].shape
assert (2,) == _stats['std'].shape

In [26]:
#export
def normalize(data, mean, std):# TODO: xxx do not touch target columns...
    column_count = len(mean) # use len rather than shape[0] so that mean stats can be lists or np arrays
    data[..., :column_count] -= mean
    data[..., :column_count] /= std
    return data

In [27]:
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
_data = select_viseme_dataset_columns(_dataset_metadata, _data, _cont_names[:2] + ['expression_id'])
_stats = calculate_stats(_data)
_normalized_data = normalize(_data, **_stats)
assert np.allclose(_normalized_data.mean(axis=0), 0)
assert np.allclose(_normalized_data.std(axis=0), 1)

In [28]:
# calculate stats and normalize without touching target - this test is like make_landmarks_relative 
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
_data = select_viseme_dataset_columns(_dataset_metadata, _data, _cont_names[:2] + ['expression_id'])
_stats = calculate_stats(_data, 2)
_normalized_data = normalize(_data, **_stats)
assert 0.0 == _normalized_data[0][2] # check that targets have not been changed
assert 4.0 == _normalized_data[4999][2] # get_image_path('20211113_114625', 'data', 1199)

In [29]:
# calculate stats and normalize without touching target - this is probably a better test
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
_data = select_viseme_dataset_columns(_dataset_metadata, _data, _cont_names[:2] + ['expression_id'])
_stats = calculate_stats(_data, 2)
assert (2,) == _stats['mean'].shape
assert (2,) == _stats['std'].shape
_normalized_data = normalize(_data, **_stats)
assert np.allclose(_normalized_data[..., :2].mean(axis=0), 0)
assert np.allclose(_normalized_data[..., :2].std(axis=0), 1)
_dataset_metadata, _data = read_viseme_dataset(test_viseme_dataset, True)
assert list(_data[..., -1]) == list(_normalized_data[..., -1])

# Combine data processing steps

In [30]:
#export
def inference_data_from_landmarks(landmarks, landmark_ids, relative_landmark_id=None, coords=['x', 'y'], stats=None):
    landmark_ids, coords, data, to_landmarks_data = sorted(landmark_ids), sorted(coords), [], []
    for landmark_id in landmark_ids:
        for coord in coords:
            if landmark_id == relative_landmark_id:
                to_landmarks_data.append(getattr(landmarks[landmark_id], coord))
            else:
                data.append(getattr(landmarks[landmark_id], coord))
    data, to_landmarks_data = np.array([data], dtype=float), np.array([to_landmarks_data], dtype=float)
    if relative_landmark_id is not None:
        make_landmarks_relative(data, to_landmarks_data)
    if stats is not None:
        normalize(data, **stats)
    return data

In [31]:
from collections import namedtuple
_Landmark = namedtuple('_Landmark', ['x','y','z'])
_landmarks=[]
for i in range(FaceLandmarks.count):
    _landmarks.append(_Landmark(i+.1, i+.2, i+.3))
assert [[-1., -1.,  1.,  1.]] == inference_data_from_landmarks(_landmarks, [0,2,1], 1).round().tolist()

In [32]:
_stats = dict(mean=[0.1,0.2,0.3,0.4], std=[0.1,0.2,0.3,0.4])
_inference_data = inference_data_from_landmarks(
        landmarks=_landmarks, landmark_ids=[2,0,1], relative_landmark_id=1, stats=_stats)
assert [[-11.0,-6.0,2.3,1.5]] == np.round(_inference_data, 1).tolist()

In [36]:
#export
def processed_dataset_from_viseme_dataset(
        input_path, landmark_ids, relative_landmark_id=None, y_name='expression_id',
        change_y_from=None, change_y_to=None): # TODO: named tuple for these 2?
    "Create a processed dataset (ready for ML) from a raw viseme dataset"
    input_path = Path(input_path)
    input_metadata, input_data = read_viseme_dataset(input_path)
    output_path = Path(input_metadata['output_path'])/f'processed_{now()}'
    output_path.mkdir()
    cont_names = landmark_ids_to_col_names(landmark_ids, relative_landmark_id)
    dataset_columns = cont_names + [y_name]
    metadata = dict(landmark_ids=landmark_ids, 
                    relative_landmark_id=relative_landmark_id, 
                    input_path=path_to_str(input_path), 
                    output_path=path_to_str(output_path),
                    y_name=y_name,
                    change_y_from=change_y_from,
                    change_y_to=change_y_to,
                    cont_names=cont_names,
                    dataset_columns=dataset_columns)
    data = select_viseme_dataset_columns(input_metadata, input_data, dataset_columns)
    if change_y_from is not None:
        assert change_y_to is not None
        data = change_target(data, change_y_from, change_y_to)
    if relative_landmark_id is not None:
        relative_names = landmark_ids_to_col_names([relative_landmark_id])
        metadata['relative_names']=relative_names
        to_landmarks = select_viseme_dataset_columns(input_metadata, input_data, relative_names)
        make_landmarks_relative(data, to_landmarks, column_count=len(cont_names))
    stats = calculate_stats(data, column_count=len(cont_names))
    normalize(data, **stats)
    with open(output_path/'metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    np.save(output_path/'data.npy', data, allow_pickle=False)
    np.savez(output_path/'stats.npz', **stats)
    return output_path

In [34]:
test_viseme_dataset

WindowsPath('data/viseme_dataset_20211115_182351')

In [37]:
_output_path = processed_dataset_from_viseme_dataset(
        input_path=test_viseme_dataset, 
        landmark_ids=FaceLandmarks.pointer + FaceLandmarks.mouth,
        relative_landmark_id=FaceLandmarks.tip_of_nose,
        change_y_from=4, change_y_to=0)

In [38]:
_output_path

WindowsPath('data/viseme_dataset_20211115_182351/processed_20211115_190237')

In [81]:
def to_processed_dataset_from_viseme_dataset_cli(
        input_path, landmark_ids, relative_landmark_id=None, y_name='expression_id',
        change_y_from=None, change_y_to=None):
    landmark_ids = '"'+','.join([str(i) for i in landmark_ids])+'"'
    print(f"""processed_dataset_from_viseme_dataset ^
        {path_to_str(input_path)} ^
        {landmark_ids} ^
        --relative_landmark_id {relative_landmark_id} ^
        --y_name {y_name} ^
        --change_y_from {change_y_from} ^
        --change_y_to {change_y_to}
    """)
to_processed_dataset_from_viseme_dataset_cli(
        input_path=test_viseme_dataset, 
        landmark_ids=FaceLandmarks.pointer + FaceLandmarks.mouth,
        relative_landmark_id=FaceLandmarks.tip_of_nose,
        change_y_from=4, change_y_to=0)

processed_dataset_from_viseme_dataset ^
        data/viseme_dataset_20211115_182351 ^
        "1,5,2,218,438,0,11,12,13,14,15,16,17,18,37,38,39,40,41,42,43,57,61,62,72,73,74,76,77,78,80,81,82,83,84,85,86,87,88,89,90,91,92,95,96,106,146,164,165,167,178,179,180,181,182,183,184,185,186,191,204,267,268,269,270,271,272,273,287,291,292,302,303,304,306,307,308,310,311,312,313,314,315,316,317,318,319,320,321,322,324,325,335,375,391,393,402,403,404,405,406,407,408,409,410,415,424" ^
        --relative_landmark_id 1 ^
        --y_name expression_id ^
        --change_y_from 4 ^
        --change_y_to 0
    


In [66]:
#export
def read_processed_dataset(dataset_path, to_numpy=True):
    "Return metadata, data and stats (as a numpy array by default, pandas dataframe if `to_numpy=False`)"
    dataset_path = Path(dataset_path)
    data, stats = np.load(dataset_path/'data.npy'), np.load(dataset_path/'stats.npz')
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    if to_numpy: 
        return dataset_metadata, data, stats
    columns = dataset_metadata['dataset_columns']
    stats = dict(mean=pd.DataFrame(stats['mean'][None, ...], columns=columns[:-1]),
                 std=pd.DataFrame(stats['std'][None, ...], columns=columns[:-1]))
    return dataset_metadata, pd.DataFrame(data, columns=columns), stats

In [85]:
# _metadata, _data, _stats = read_processed_dataset(_output_path)
_metadata, _data, _stats = read_processed_dataset('data/viseme_dataset_20211115_182351/processed_20211115_193955', False)

In [86]:
_data

,0x,0y,1x,1y,106x,106y,11x,11y,12x,12y,...,90y,91x,91y,92x,92y,95x,95y,96x,96y,expression_id
0,0.507680,0.523119,0.498190,0.499518,0.509128,0.504540,0.503329,0.502060,0.507636,0.524137,...,0.507862,0.524789,0.510128,0.526690,0.505256,0.523396,0.493935,0.507482,0.500756,0.0
1,0.511360,0.521539,0.498190,0.499518,0.510243,0.503171,0.502309,0.500360,0.511837,0.522522,...,0.508325,0.525190,0.508664,0.526685,0.505597,0.527920,0.492171,0.507456,0.499468,0.0
2,0.507055,0.520180,0.498190,0.499518,0.507756,0.504848,0.502999,0.499940,0.507040,0.521072,...,0.508964,0.519579,0.510436,0.519101,0.506961,0.518297,0.496197,0.508319,0.500151,0.0
3,0.502883,0.520394,0.498190,0.499518,0.505290,0.502473,0.502009,0.502271,0.502524,0.521261,...,0.506327,0.514473,0.506702,0.513345,0.503990,0.514418,0.490172,0.507765,0.498145,0.0
4,0.503280,0.516075,0.498190,0.499518,0.504235,0.503401,0.501365,0.501225,0.502995,0.516551,...,0.503497,0.510866,0.505514,0.512032,0.501362,0.507551,0.489326,0.504007,0.500237,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-1.750383,-2.110532,-1.749757,-2.121363,-1.749918,-2.117776,-1.749327,-2.130634,-1.750449,-2.109069,...,-2.107131,-1.776180,-2.109321,-1.766386,-2.106749,-1.774810,-2.099320,-1.763398,-2.122368,0.0
4996,-1.741017,-2.102960,-1.740357,-2.113045,-1.740530,-2.109704,-1.739912,-2.121668,-1.741087,-2.101598,...,-2.099773,-1.767795,-2.101810,-1.757647,-2.099416,-1.766398,-2.092451,-1.754500,-2.113977,0.0
4997,-1.721032,-2.099915,-1.720299,-2.109699,-1.720499,-2.106457,-1.719821,-2.118061,-1.721111,-2.098593,...,-2.096814,-1.749902,-2.098790,-1.739001,-2.096467,-1.748448,-2.089687,-1.735514,-2.110602,0.0
4998,-1.711363,-2.101433,-1.710596,-2.111367,-1.710807,-2.108075,-1.710102,-2.119858,-1.711447,-2.100091,...,-2.098289,-1.741246,-2.100295,-1.729981,-2.097937,-1.739763,-2.091065,-1.726329,-2.112284,0.0


TODO: run this code as a test when we have a small sample of test session data

we created "viseme_dataset_20211019_000000" with
```
_output_path = processed_dataset_from_viseme_dataset(
        dataset_id='20211019_000000', 
        landmark_ids=FaceLandmarks.pointer + FaceLandmarks.mouth,
        relative_landmark_id=FaceLandmarks.tip_of_nose,
        path='test/data')
```
we will problably do something like ...
```
change_from_id=4, change_to_id=0
```
... but we won't be able to reconstruct "raw" data if we do

In [ ]:
# _output_path = Path('test/data/viseme_dataset_20211019_000000/processed_20211024_000000')
_processed_data = np.load(_output_path/'data.npy')
assert (2000, 213) == _processed_data.shape
assert np.allclose(_processed_data[...,:-1].mean(axis=0), 0)
assert np.allclose(_processed_data[...,:-1].std(axis=0), 1)

In [ ]:
with open(_output_path/'metadata.json') as f:
    _processed_metadata = json.load(f)

In [ ]:
df

In [ ]:
pd.DataFrame(_processed_data, columns=_processed_metadata['dataset_columns'])

In [ ]:
_processed_stats = np.load(_output_path/'stats.npz')
assert ['mean', 'std'] == list(_processed_stats.keys())
# TODO: more detailed tests

In [ ]:
# TODO: fix tests
# with open(_output_path/'metadata.json') as f:
#     _processed_metadata = json.load(f)
# assert _dataset_id == _processed_metadata['dataset_id']
# assert FaceLandmarks.pointer + FaceLandmarks.mouth == _processed_metadata['landmark_ids']
# assert FaceLandmarks.tip_of_nose == _processed_metadata['relative_landmark_id']
# assert _y_name == _processed_metadata['y_name']
# assert _cont_names == _processed_metadata['cont_names']
# assert ['1x','1y'] == _processed_metadata['relative_names']

In [ ]:
_processed_data = _processed_data * _processed_stats['std']
_processed_data = _processed_data + _processed_stats['mean']
_, _relative_data = read_viseme_dataset('20211019_000000', 'test/data', ['1x','1y'])
for i in range(2):
    _processed_data[:, i:212:2] += _relative_data[..., i][..., None]
_df = pd.DataFrame(_processed_data, columns=_processed_metadata['cont_names'] + [_processed_metadata['y_name']])
_, _raw_df = read_viseme_dataset(_dataset_id, _path, _processed_metadata['cont_names'], _processed_metadata['y_name'], to_numpy=False)
_df, _raw_df = _df.round(2), _raw_df.round(2)
_df['expression_id'] = _df['expression_id'].astype(int)
_raw_df['expression_id'] = _raw_df['expression_id'].astype(int)
assert _df.equals(_raw_df), 'we should be able to reconstruct the "raw" data from "processed"'

In [75]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 05a_ml_data.ipynb.
Converted 05b_ml_model.ipynb.
Converted 10a_mediapipe_face_mesh_identify_landmarks.ipynb.
Converted 10b_mediapipe_face_mesh_capture_session.ipynb.
Converted 10c_mediapipe_face_mesh_train_model.ipynb.
Converted 10d_test_np_model.ipynb.
Converted 20a_gui_capture_command.ipynb.
Converted 70_cli.ipynb.
Converted index.ipynb.
Converted project_lifecycle.ipynb.
